https://pypi.org/project/scapy-unroot/  
https://pypi.org/project/lanscan/  
   
https://pypi.org/project/netifaces/  
https://pypi.org/project/get-mac/  
https://github.com/giampaolo/psutil  

https://github.com/devmarcstorm/python-network-scanner https://pypi.org/project/python-nmap/  
    - https://geektechstuff.com/2020/06/03/python-and-nmap-scanning-for-hosts-python/  
    - https://www.studytonight.com/network-programming-in-python/integrating-port-scanner-with-nmap  

    

#### scapy based
https://github.com/jtalway/Network-Scanner  
https://github.com/dharmil18/Network-Scanner  

#### don't work on macos  
https://pypi.org/project/networkscan/  


In [ ]:

'''
## Reference

https://stackoverflow.com/a/166589
'''



In [ ]:

import socket, subprocess
from getmac import get_mac_address
import netifaces



In [13]:

## This Machine Local IP
def get_this_device_mapping(way=1):
    '''
    Returns:
        {ip:mac_address} of this device
    '''
    if way==1:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        this_device_local_ip = s.getsockname()[0]
        s.close()
        this_device_mac = get_mac_address(ip=this_device_local_ip)
    
    elif way==2:
        # netifaces.interfaces()
        address = netifaces.ifaddresses('en0')
        this_device_local_ip = address[netifaces.AF_INET][0]['addr']
        this_device_mac = address[netifaces.AF_LINK][0]['addr']

    local_network_map_di = {
        this_device_local_ip: this_device_mac
    }
    return local_network_map_di


get_this_device_mapping()


{'192.168.1.45': '38:f9:d3:c2:39:da'}

In [15]:

## get all IP that are connected to router (Local Network)
def get_local_network_mapping():
    ''' all the IPs starting with 192.168.1'''
    # for i in range(255):
    #     command=['ping', '-n', '1','-w','100', '192.168.1.'+str(i)]
    #     print(' '.join(command))
    #     subprocess.call(command)

    arpa = subprocess.check_output(('arp', '-a')).decode('ascii')
    li = [ (e.split()[1], e.split()[3]) for e in [ rec for rec in arpa.split('\n') ] if len(e.split()) > 3]
    mapping_di = { ''.join([ e for e in e1 if e.isdigit() or e=='.']):e2 
                  for e1,e2 in li if 'incomplete' not in e2 }
    # mapping_di = { k:mapping_di[k] for k in mapping_di if '192.168.1' in k }
    return mapping_di



get_local_network_mapping()



{'192.168.1.1': 'b4:f9:49:48:5a:d0',
 '192.168.1.45': '38:f9:d3:c2:39:da',
 '192.168.1.255': 'ff:ff:ff:ff:ff:ff',
 '224.0.0.251': '1:0:5e:0:0:fb',
 '239.255.255.250': '1:0:5e:7f:ff:fa'}

In [ ]:
# rtsp://username:password@IP Address:554/stream1


In [ ]:
get_this_device_mapping()

In [ ]:
nmap -sP 192.168.1.0/24 | grep -B 2 "a6:94:15:5e:36:0" | head -n 1 | cut -d " " -f 5

In [7]:
## main

# from prettytable import PrettyTable
# import json

import os
import sys
import nmap
from getmac import get_mac_address
from datetime import date, datetime



In [8]:
## network

nm = nmap.PortScanner()

scan_range = nm.scan(hosts="192.168.1.2", timeout=100)

print(scan_range['scan'])

    
    

{}


In [9]:
# from network import Network

class Network(object):
    def __init__(self, ip=None):
        self.ip_default = '192.168.1.1'
        self.ip = ip if ip is not None else self.ip_default
        self.nm = nmap.PortScanner()
        
        
    def get_devices(self):
        '''
        Desc: Creates a list of items that contain device information
        Return:
            a list
        '''
        if len(self.ip) >= 1:
            network_to_scan = self.ip + '/24'
        else:
            network_to_scan = self.ip_default + '/24'

        print('Scanning {}...'.format(network_to_scan))
        self.nm.scan(hosts=network_to_scan, arguments='-sn') 
        device_list = [(device, self.nm[device]) for device in self.nm.all_hosts()]
        return device_list
    

devices = Network('192.168.1.10').get_devices()

for host, info in devices:
    info['mac'] = get_mac_address(ip=host)
    

devices


Scanning 192.168.1.10/24...


[('192.168.1.1',
  {'hostnames': [{'name': '', 'type': ''}],
   'addresses': {'ipv4': '192.168.1.1'},
   'vendor': {},
   'status': {'state': 'up', 'reason': 'syn-ack'},
   'mac': 'b4:f9:49:48:5a:d0'})]

In [10]:
class Device(object):
    def __init__(self, mac, ip, network_name, data={}):
        
        self.color_di = { # terminal color
            'red': '\033[0;31;40m',
            'green': '\033[0;32;40m',
            'normal': '\033[0m'
        }
        self.mac = mac
        self.ip = ip
        self.network_name = network_name

        self.name = None
        self.allowed = None
        self.location = None

        device_check = self.device_known(data)
        if device_check:
            self.name = device_check['name']
            self.allowed = device_check['allowed']
            self.location = device_check['location']

    def device_known(self, data):
        '''Return a str (given name of the device) or None

        Checks whether the device is contained in the Dictonary (based on the mac address)
        '''
        mac = ''
        if self.mac in data:
            mac = self.mac
        elif self.mac.upper() in data:
            mac = self.mac.upper()

        if mac:
            name = '{} of {}'.format(data[mac]['type'], data[mac]['owner'])
            return {'name': name, 'allowed': data[mac]['allowed'], 'location': data[mac]['location']}

        return None

    def to_list(self):
        '''Return a list [mac, ip, network_name, name, location, allowed]

        Creates a list of device attributes as colored strings
            green: allowed in the network
            red: not allowed
        '''
        if self.allowed:
            color = g
        else:
            color = r

        mac = '{}{}{}'.format(color, self.mac, n)
        ip = '{}{}{}'.format(color, self.ip, n)
        network_name = '{}{}{}'.format(color, self.network_name, n)
        name = '{}{}{}'.format(color, self.name, n)
        location = '{}{}{}'.format(color, self.location, n)
        allowed = '{}{}{}'.format(color, self.allowed, n)

        return [mac, ip, network_name, name, location, allowed]

    def to_string(self):
        '''Return a str

        Device information as a string with indentations for the log file
        '''
        return 'Log: {} \n\t Mac Address: {} \n\t Name in network: {} \n\t Given name: {} \n\t Allowed on network: {}'.format(datetime.now(), self.mac, self.network_name, self.name, self.allowed)


In [ ]:





# from device import Device




    



def create_device_list(devices, data):
    ''' Return a dictonary like {'known': [], 'unknown': []}

    Creates 2 lists from devices (class Device) and makes them available in a dictionary
       - 'known': list of known devices (mac address included in the data/device.json)
       - 'unknown': list of unknown devices (not included)
    '''
    known_devices = []
    unknown_devices = []

    for host, info in devices:
        device = Device(info['mac'], host, info['hostnames'][0]['name'], data)
        if device.name:
            known_devices.append(device)
        else:
            unknown_devices.append(device)

    return {'known': known_devices, 'unknown': unknown_devices}

if __name__ == '__main__':
    dataPath = 'data'
    try:
        with open("{}/devices.json".format(dataPath), "r") as readFile:
                json_devices = json.load(readFile)
    except FileNotFoundError:
                json_devices = {}
                print('''No valid "data/devices.json" found. Please create one with the following format:
{
    "00:00:00:00:00:00":
    {
      "type": "Device",
      "owner": "John Appleseed",
      "location": null,
      "allowed": true
    }
}
            ''')

    network = Network()

    try:
        devices = network.get_devices()
    except KeyboardInterrupt:
        print('You stopped scanning. Scanning may take a while. If it takes too long, there may be a problem with the connection. Did you specify the correct network?')
        sys.exit()

    for host, info in devices:
        info['mac'] = get_mac_address(ip=host)

    data = create_device_list(devices, json_devices)
    log_text = ''

    table = PrettyTable()
    table.field_names = ["MAC ADDRESS", "IP", "NAME IN NETWORK", "NAME", 'LOCATION', 'ALLOWED']
    for device in data['known']:
        table.add_row(device.to_list())
        log_text += '{}\n'.format(device.to_string())
    
    print('Known Devices\n{}'.format(table))

    table = PrettyTable()
    table.field_names = ["MAC ADDRESS", "IP", "NAME IN NETWORK"]
    for device in data['unknown']:
        table.add_row(device.to_list()[:3])
        log_text += '{}\n'.format(device.to_string())
    
    print('Unknown Devices\n{}'.format(table))

if not os.path.isdir(dataPath):
    os.mkdir(dataPath)

with open("{}/{}.log".format(dataPath, date.today()), "a") as appendFile:
    appendFile.write(log_text)
    print('You can find a log file with all devices in "data/{}.log"'.format(date.today()))


---

## Other

---

In [ ]:
## Getting MAC address of this device





In [ ]:
import uuid
print(hex(uuid.getnode()))


def get_mac_address(setting='en0'):
    """Get mac address.
    """
    # mac = uuid.UUID(int=uuid.getnode()).hex[-12:]
    # return ":".join([mac[e:e+2] for e in range(0, 11, 2)]) 
    if setting=='en0':
        return ':'.join(re.findall('..', '%012x' % uuid.getnode())) 
    return ':'.join(['{:02x}'.format((uuid.getnode() >> elements) & 0xff) 
                            for elements in range(0,2*6,2)][::-1])


get_mac_address()


In [ ]:
import netifaces

netifaces.interfaces()
netifaces.ifaddresses('en0')

In [ ]:
netifaces.ifaddresses('en5')


In [ ]:
address = netifaces.ifaddresses('en0')
address

In [ ]:
address[netifaces.AF_INET][0]['addr']
address[netifaces.AF_LINK][0]['addr']



In [ ]:
import socket
import psutil

def get_ip_addresses(family):
    for interface, snics in psutil.net_if_addrs().items():
        for snic in snics:
            if snic.family == family:
                yield (interface, snic.address)

ipv4s = list(get_ip_addresses(socket.AF_INET))
ipv6s = list(get_ip_addresses(socket.AF_INET6))

ipv4s, list(get_ip_addresses(socket.AddressFamily))


In [ ]:


netifaces.ifaddresses('lo0')


In [ ]:


netifaces.AF_LINK


addrs = netifaces.ifaddresses('lo0')
addrs[netifaces.AF_INET]


In [ ]:
addrs = 
addrs[netifaces.AF_INET]


In [ ]:
addrs[netifaces.AF_LINK]


In [ ]:
addrs = netifaces.ifaddresses('fw0')
addrs[netifaces.AF_LINK]


In [ ]:
# from getmac import get_mac_address
# eth_mac = get_mac_address(interface="eth0")
# win_mac = get_mac_address(interface="Ethernet 3")
# ip_mac = get_mac_address(ip="192.168.0.1")
# ip6_mac = get_mac_address(ip6="::1")
# host_mac = get_mac_address(hostname="localhost")
# updated_mac = get_mac_address(ip="10.0.0.1", network_request=True)

# # Changing the port used for updating ARP table (UDP packet)
# from getmac import getmac
# getmac.PORT = 44444  # Default: 55555
# print(getmac.get_mac_address(ip="192.168.0.1", network_request=True))

# # Enabling debugging
# from getmac import getmac
# getmac.DEBUG = 2  # DEBUG level 2
# print(getmac.get_mac_address(interface="Ethernet 3"))

https://www.thepythoncode.com/article/building-wifi-scanner-in-python-scapy

-----